In [ ]:
#grid_walker.ipynb
#
#by Joe Hahn
#jmh.datasciences@gmail.com
#3 February 2018
#
#grid_walker uses Q-learning to teach a neural net AI how to navigate an agent about
#a 6x6 grid, guiding it towards a goal while avoiding obstacles and hazards

In [ ]:
#import plotting libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, font='DejaVu Sans')

In [ ]:
#game settings
grid_size = 6
init = 'random_agent'
rn_seed = 15

In [ ]:
#seed random number generator
import numpy as np
np.random.seed(rn_seed)

In [ ]:
from grid_walker import *

In [ ]:
#check initial conditions
environment = initialize_environment(grid_size, init)
state = initialize_state(environment)
objects = environment['objects']
actions = environment['actions']
acts = environment['acts']
state_vector = state2vector(state, environment)
grid = make_grid(state, environment)
print 'objects = ', objects
print 'actions = ', actions
print 'acts = ', acts
print 'state = ', state
print 'state_vector = ', state_vector
print 'state_vector.shape = ', state_vector.shape
print np.rot90(grid.T)
N_inputs = state_vector.shape[1]
N_outputs = len(actions)
grid_size = environment['grid_size']
max_moves = environment['max_moves']
print 'N_inputs = ', N_inputs
print 'N_outputs = ', N_outputs
print 'grid_size = ', grid_size
print 'max_moves = ', max_moves
print 'rn_seed = ', rn_seed

In [ ]:
#assemble neural network that agent will use to navigate the grid
model = build_model(N_inputs, grid_size, N_outputs)
print model.summary()

In [ ]:
#train neural network to walk agent towards goal
N_training_games = 500                #number of games to play while training model
gamma = 0.9                           #discount for future rewards
memories_size = 100                   #size of memory queue size, for experience replay
batch_size = memories_size/5          #number of memories to use when retraining the model
debug = True
trained_model = train(model, N_training_games, gamma, memories_size, batch_size, debug=debug)

In [ ]:
#test the trained neural network
initial_state, final_state, N_moves = test_model(trained_model, environment)